In [1]:
#karas install
!pip install keras

                                              0.0/1.7 MB ? eta -:--:--
                                              0.0/1.7 MB ? eta -:--:--
     -                                        0.1/1.7 MB 787.7 kB/s eta 0:00:03
     -------------------------                1.1/1.7 MB 8.8 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 11.0 MB/s eta 0:00:00


In [3]:
# tf install
!pip install tensorflow

                                              0.0/272.9 MB ? eta -:--:--
                                              0.0/272.9 MB ? eta -:--:--
                                              0.1/272.9 MB 1.8 MB/s eta 0:02:34
                                             1.2/272.9 MB 12.3 MB/s eta 0:00:23
                                             2.2/272.9 MB 15.8 MB/s eta 0:00:18
                                             3.3/272.9 MB 17.7 MB/s eta 0:00:16
                                             4.4/272.9 MB 18.9 MB/s eta 0:00:15
                                             5.5/272.9 MB 19.5 MB/s eta 0:00:14
                                             6.6/272.9 MB 20.0 MB/s eta 0:00:14
     -                                       7.6/272.9 MB 20.2 MB/s eta 0:00:14
     -                                       8.7/272.9 MB 20.6 MB/s eta 0:00:13
     -                                       9.8/272.9 MB 20.8 MB/s eta 0:00:13
     -                                      10.8/272.9

In [6]:
# MNIST data load
from keras.datasets import mnist
X, y = mnist.load_data()[0]
X.shape, y.shape

((60000, 28, 28), (60000,))

In [15]:
import pickle as pkl
data = mnist.load_data()
with open(r'./data/mnist.pkl', 'wb') as f:
    pkl.dump(data, f)

In [31]:
# 데이터셋 나누기
from sklearn.model_selection import train_test_split
X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size=0.166666, random_state=1473032201)
X_tmp.shape, X_test.shape, y_tmp.shape, y_test.shape

((50000, 28, 28), (10000, 28, 28), (50000,), (10000,))

In [50]:
(X_tmp, y_tmp),(X_test, y_test) = mnist.load_data()

In [51]:
X_tmp.shape, y_tmp.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [52]:
import numpy as np
np.unique(y_tmp, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949],
       dtype=int64))

In [53]:
# 28*28 -> 784
X_tmp = X_tmp.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_tmp.shape, X_test.shape

((60000, 784), (10000, 784))

In [54]:
# 0~255 -> 0~1 : 정규화
X_tmp = X_tmp.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [56]:
# stratified k-fold
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1473032201)
skf.get_n_splits(X_tmp, y_tmp)

5

In [61]:
# tree clf 모델 생성
# accuracy 스코어 출력
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
scores = []
max_d_range = range(10, 21)
for depth in max_d_range:
    depth_scores = []
    clf = DecisionTreeClassifier(max_depth=depth, random_state=1473032201)
    for train_index, test_index in skf.split(X_tmp, y_tmp):
        X_train, X_valid = X_tmp[train_index], X_tmp[test_index]
        y_train, y_valid = y_tmp[train_index], y_tmp[test_index]
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_valid)
        score = accuracy_score(y_valid, y_pred)
        depth_scores.append(score)
    mean_score = np.mean(depth_scores)
    scores.append((mean_score, depth))

In [62]:
scores.sort(key=lambda x: x[0], reverse=True)
scores

[(0.86995, 18),
 (0.8695999999999999, 14),
 (0.8692333333333334, 15),
 (0.8692333333333332, 19),
 (0.8692, 20),
 (0.8688666666666667, 16),
 (0.8684666666666667, 13),
 (0.8682000000000001, 17),
 (0.8674666666666667, 12),
 (0.8607333333333334, 11),
 (0.8523833333333334, 10)]